## Load Data

In [62]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List
import json

# **** SETUP ****

# change to match your filesystem
DATA_DIR = "./data/air_travel/"
DEFAULT_RECEIPTS_FILE = os.path.join(DATA_DIR, "tickets.json")
PROJECT_NAME = "deb-dev-dw"
DATASET_NAME = "air_travel"

data = []
with open('./data/air_travel/tickets.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame.from_dict(pd.json_normalize(data), orient='columns')

display(df.head(n=10))

,eticket_num,confirmation,ticket_date,price,seat,status,airline.name,airline.iata,airline.icao,airline.callsign,...,passenger.last_name,passenger.gender,passenger.birth_date,passenger.email,passenger.street,passenger.city,passenger.state,passenger.zip,origin,destination
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,China Eastern Airlines,MU,CES,CHINA EASTERN,...,Brown,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN
1,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,Hawaiian Airlines,HA,HAL,HAWAIIAN,...,Kent,F,1998-08-05,laura.kent.98@hotmail.com,13991 Davis Village,North Catherineborough,PA,16516,NaN,NaN
2,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,Wizz Air,W6,WZZ,WIZZ AIR,...,Tucker,F,1965-01-22,lisa.tucker.65@hotmail.com,04135 Marvin Via,North Kristabury,MA,01093,NaN,NaN
3,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,AirAsia,AK,AXM,ASIAN EXPRESS,...,Yates,NB,1975-03-31,matthew.yates.75@yahoo.com,76045 Samantha Road Suite 111,Lake Jeffrey,DE,19898,NaN,NaN
4,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,Xiamen Airlines,MF,CXA,XIAMEN AIR,...,Villanueva,NB,1945-08-14,megan.villanueva.45@hotmail.com,848 Melissa Springs Suite 947,Kellerstad,TX,76177,NaN,NaN
5,115-196069-8963,XFYQC0,2022-03-23,892.69,18C,active,Air New Zealand,NZ,ANZ,NEW ZEALAND,...,Hall,NB,1944-08-31,sarah.hall.44@gmail.com,75420 Michael Mountains Suite 485,New Victoria,HI,96727,NaN,NaN
6,396-673460-1326,N5UOOZ,2022-03-23,889.53,3C,active,Jeju Air,7C,JJA,JEJU AIR,...,Thompson,M,1968-05-02,seth.thompson.68@yahoo.com,22455 Higgins Junction Apt. 042,New Keith,OR,97405,NaN,NaN
7,380-894599-8109,PAA19Y,2022-03-22,706.78,7D,active,American Airlines,AA,AAL,AMERICAN,...,Garcia,F,1950-02-12,jennifer.garcia.50@gmail.com,6607 Sharp Common,Chadstad,VA,22121,NaN,NaN
8,614-960971-2686,EF4BHJ,2022-03-23,486.40,24J,active,Juneyao Airlines,HO,DKH,JUNEYAO AIRLINES,...,Clark,F,1991-11-09,becky.clark.91@gmail.com,691 Jones Cliffs,Michaelburgh,TX,76003,NaN,NaN
9,481-321233-0702,FVM9EE,2022-03-23,855.93,16A,active,Royal Air Maroc,AT,RAM,ROYALAIR MAROC,...,Cook,M,1976-07-29,ronald.cook.76@hotmail.com,93328 Davis Island,Rodriguezside,MD,21408,NaN,NaN


In [63]:
# modify column name to work better with big query
df.columns = df.columns.str.replace(".", "_")
display(df.head(n=10))

/Users/drewwhite/Desktop/Epicodus/data_week_8/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,eticket_num,confirmation,ticket_date,price,seat,status,airline_name,airline_iata,airline_icao,airline_callsign,...,passenger_last_name,passenger_gender,passenger_birth_date,passenger_email,passenger_street,passenger_city,passenger_state,passenger_zip,origin,destination
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,China Eastern Airlines,MU,CES,CHINA EASTERN,...,Brown,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN
1,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,Hawaiian Airlines,HA,HAL,HAWAIIAN,...,Kent,F,1998-08-05,laura.kent.98@hotmail.com,13991 Davis Village,North Catherineborough,PA,16516,NaN,NaN
2,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,Wizz Air,W6,WZZ,WIZZ AIR,...,Tucker,F,1965-01-22,lisa.tucker.65@hotmail.com,04135 Marvin Via,North Kristabury,MA,01093,NaN,NaN
3,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,AirAsia,AK,AXM,ASIAN EXPRESS,...,Yates,NB,1975-03-31,matthew.yates.75@yahoo.com,76045 Samantha Road Suite 111,Lake Jeffrey,DE,19898,NaN,NaN
4,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,Xiamen Airlines,MF,CXA,XIAMEN AIR,...,Villanueva,NB,1945-08-14,megan.villanueva.45@hotmail.com,848 Melissa Springs Suite 947,Kellerstad,TX,76177,NaN,NaN
5,115-196069-8963,XFYQC0,2022-03-23,892.69,18C,active,Air New Zealand,NZ,ANZ,NEW ZEALAND,...,Hall,NB,1944-08-31,sarah.hall.44@gmail.com,75420 Michael Mountains Suite 485,New Victoria,HI,96727,NaN,NaN
6,396-673460-1326,N5UOOZ,2022-03-23,889.53,3C,active,Jeju Air,7C,JJA,JEJU AIR,...,Thompson,M,1968-05-02,seth.thompson.68@yahoo.com,22455 Higgins Junction Apt. 042,New Keith,OR,97405,NaN,NaN
7,380-894599-8109,PAA19Y,2022-03-22,706.78,7D,active,American Airlines,AA,AAL,AMERICAN,...,Garcia,F,1950-02-12,jennifer.garcia.50@gmail.com,6607 Sharp Common,Chadstad,VA,22121,NaN,NaN
8,614-960971-2686,EF4BHJ,2022-03-23,486.40,24J,active,Juneyao Airlines,HO,DKH,JUNEYAO AIRLINES,...,Clark,F,1991-11-09,becky.clark.91@gmail.com,691 Jones Cliffs,Michaelburgh,TX,76003,NaN,NaN
9,481-321233-0702,FVM9EE,2022-03-23,855.93,16A,active,Royal Air Maroc,AT,RAM,ROYALAIR MAROC,...,Cook,M,1976-07-29,ronald.cook.76@hotmail.com,93328 Davis Island,Rodriguezside,MD,21408,NaN,NaN


## Define dimension table schema

In [65]:
# setting table schema for dims tables

DIMS_TABLE_METADATA = {  
    # for airlines
    'airlines': {
        'table_name': 'airlines',
        'schema': [
            bigquery.SchemaField('airline_iata', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('airline_name', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('airline_icao', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('airline_callsign', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('airline_country', 'STRING', mode='NULLABLE'),
        ],
    },
    # for airports
    'airports': {
        'table_name': 'airports',
        'schema': [
            bigquery.SchemaField('airport_iata', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('airport_name', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('airport_city', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('airport_icao', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('airport_latitude', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('airport_longitude', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('airport_altitude', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('airport_tz_timezone', 'STRING', mode='NULLABLE'),
        ],
    },
    # for passengers
    'passengers': {
        'table_name': 'passengers',
        'schema': [
            bigquery.SchemaField('passenger_id', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('passenger_email', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('passenger_first_name', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('passenger_last_name', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('passenger_birth_date', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('passenger_street', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('passenger_city', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('passenger_state', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('passenger_zip', 'INTEGER', mode='NULLABLE'),
            bigquery.SchemaField('effective_start_date', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('effective_end_date', 'STRING', mode='NULLABLE'),
        ],
    },
}

# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")
client = bigquery.Client()

logger.info(f"Setup Completed")

[DEBUG][2023-01-06 14:03:34,309][4002363161:0060] : Creating bigquery client
[INFO ][2023-01-06 14:03:34,313][4002363161:0063] : Setup Completed


## Create dataset in bigquery

In [66]:

dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)
# tells me that dataset was created with name
logger.info(f"Created air_travel dataset: {dataset.full_dataset_id}")

[INFO ][2023-01-06 14:03:38,386][1320071499:0006] : Created air_travel dataset: deb-dev-dw:air_travel


## Load original data frame into separate data frames to make manipulations for dimensions

In [67]:
# airlines dimension table
airlines_df = df
# columns to keep for dimension
airline_expected_cols = ['airline_iata', 'airline_name', 'airline_icao', 'airline_callsign', 'airline_country']
# groupby above columns
airlines_df = airlines_df.groupby(airline_expected_cols).all()
airlines_df = airlines_df.reset_index().loc[:, airline_expected_cols]

logger.info(f"airlines - found {len(airlines_df.index)} rows")

display(airlines_df)


[INFO ][2023-01-06 14:03:40,690][1400822048:0009] : airlines - found 48 rows


,airline_iata,airline_name,airline_icao,airline_callsign,airline_country
0,3U,Sichuan Airlines,CSC,SI CHUAN,China
1,7C,Jeju Air,JJA,JEJU AIR,Republic of Korea
2,9K,Cape Air,KAP,CAIR,United States
3,9S,Spring Airlines,CQH,AIR SPRING,China
4,AA,American Airlines,AAL,AMERICAN,United States
5,AC,Air Canada,ACA,AIR CANADA,Canada
6,AF,Air France,AFR,AIRFRANS,France
7,AK,AirAsia,AXM,ASIAN EXPRESS,Malaysia
8,AS,Alaska Airlines,ASA,Inc.,ALASKA
9,AT,Royal Air Maroc,RAM,ROYALAIR MAROC,Morocco


In [68]:
# airports dimension table

origin_df = df
dest_df = df

airport_expected_cols = ['name', 'city','country', 'iata', 'icao','latitude','longitude','altitude', 'tz_timezone']

origin_col = ['origin_' + col for col in airport_expected_cols]
dest_col = ['destination_' + col for col in airport_expected_cols]
airport_col = ['airport_' + col for col in airport_expected_cols]

# group unique origin airports
origin_df = origin_df.groupby(origin_col).all()
origin_df = origin_df.reset_index().loc[:, origin_col]

# rename columns

origin_df = origin_df.rename(columns={
    'origin_name': 'airport_name', 
    'origin_city':'airport_city', 
    'origin_country':'airport_country',
    'origin_iata':'airport_iata',
    'origin_icao':'airport_icao',
    'origin_latitude':'airport_latitude',
    'origin_longitude':'airport_longitude',
    'origin_altitude':'airport_altitude',
    'origin_tz_timezone':'airport_tz_timezone',
    })

# group unique destination airports
dest_df = dest_df.groupby(dest_col).all()
dest_df = dest_df.reset_index().loc[:, dest_col]

# rename columns

dest_df = dest_df.rename(columns={
    'destination_name': 'airport_name', 
    'destination_city':'airport_city', 
    'destination_country':'airport_country',
    'destination_iata':'airport_iata',
    'destination_icao':'airport_icao',
    'destination_latitude':'airport_latitude',
    'destination_longitude':'airport_longitude',
    'destination_altitude':'airport_altitude',
    'destination_tz_timezone':'airport_tz_timezone',
    })

# combine origin and destination
concat_df = [origin_df, dest_df]

airports_df = pd.concat(concat_df)

# group unique combined airports data

airports_df = airports_df.groupby(airport_col).all()
airports_df = airports_df.reset_index().loc[:, airport_col]

logger.info(f"airports - found {len(airports_df.index)} rows")

display(airports_df)


[INFO ][2023-01-06 14:03:44,011][4159366420:0058] : airports - found 386 rows


,airport_name,airport_city,airport_country,airport_iata,airport_icao,airport_latitude,airport_longitude,airport_altitude,airport_tz_timezone
0,Abu Dhabi International Airport,Abu Dhabi,United Arab Emirates,AUH,OMAA,24.43,54.65,88.0,Asia/Dubai
1,Adolfo Suárez Madrid–Barajas Airport,Madrid,Spain,MAD,LEMD,40.47,-3.56,1998.0,Europe/Madrid
2,Afonso Pena Airport,Curitiba,Brazil,CWB,SBCT,-25.53,-49.18,2988.0,America/Sao_Paulo
3,Alberto Alcolumbre Airport,Macapa,Brazil,MCP,SBMQ,0.05,-51.07,56.0,America/Fortaleza
4,Albuquerque International Sunport,Albuquerque,United States,ABQ,KABQ,35.04,-106.61,5355.0,America/Denver
...,...,...,...,...,...,...,...,...,...
381,Zhuhai Jinwan Airport,Zhuhai,China,ZUH,ZGSD,22.01,113.38,23.0,Asia/Shanghai
382,Zumbi dos Palmares Airport,Maceio,Brazil,MCZ,SBMO,-9.51,-35.79,387.0,America/Fortaleza
383,Zürich Airport,Zurich,Switzerland,ZRH,LSZH,47.46,8.55,1416.0,Europe/Zurich
384,Ástor Piazzola International Airport,Mar Del Plata,Argentina,MDQ,SAZM,-37.93,-57.57,72.0,America/Buenos_Aires


In [69]:
import uuid

pass_df = df

pass_expected_cols = ['email','first_name','last_name', 'birth_date', 'street','city','state','zip']
pass_col = ['passenger_' + col for col in pass_expected_cols]

# group unique passengers
pass_df = pass_df.groupby(pass_col).all()
pass_df = pass_df.reset_index().loc[:, pass_col]

# generate uuid for passenger id
pass_df.insert(0, 'passenger_id', uuid.uuid4())
pass_df = pass_df.astype({'passenger_id': str, 'passenger_zip': int})

# add start and end dates
pass_df['effective_start_date'] = "2006-06-06"
pass_df['effective_end_date'] = None

display(pass_df)



,passenger_id,passenger_email,passenger_first_name,passenger_last_name,passenger_birth_date,passenger_street,passenger_city,passenger_state,passenger_zip,effective_start_date,effective_end_date
0,84a51f9a-c07f-4b07-b426-af9f6e6142e4,annette.hawkins.43@yahoo.com,Annette,Hawkins,1943-07-11,361 Robinson Green Apt. 635,North Lynntown,NV,89825,2006-06-06,None
1,84a51f9a-c07f-4b07-b426-af9f6e6142e4,autumn.morse.60@hotmail.com,Autumn,Morse,1960-01-18,6984 Price Shoals,Erictown,HI,96818,2006-06-06,None
2,84a51f9a-c07f-4b07-b426-af9f6e6142e4,becky.clark.91@gmail.com,Becky,Clark,1991-11-09,691 Jones Cliffs,Michaelburgh,TX,76003,2006-06-06,None
3,84a51f9a-c07f-4b07-b426-af9f6e6142e4,belinda.cook.91@hotmail.com,Belinda,Cook,1991-01-26,1965 Kelly Field Apt. 094,Jonesberg,IL,60613,2006-06-06,None
4,84a51f9a-c07f-4b07-b426-af9f6e6142e4,carl.wilson.80@hotmail.com,Carl,Wilson,1980-04-24,2814 Houston Hills,Rodriguezside,IA,51971,2006-06-06,None
5,84a51f9a-c07f-4b07-b426-af9f6e6142e4,cheryl.hughes.45@gmail.com,Cheryl,Hughes,1945-05-20,00992 Garcia Plaza Suite 367,North Chelseamouth,CT,6315,2006-06-06,None
6,84a51f9a-c07f-4b07-b426-af9f6e6142e4,christian.stevenson.93@hotmail.com,Christian,Stevenson,1993-06-14,75945 Jennifer Loaf,Pooleland,KY,40009,2006-06-06,None
7,84a51f9a-c07f-4b07-b426-af9f6e6142e4,corey.cook.83@gmail.com,Corey,Cook,1983-06-14,9606 Barton Station Apt. 271,Jacquelinemouth,IN,47081,2006-06-06,None
8,84a51f9a-c07f-4b07-b426-af9f6e6142e4,danielle.henderson.70@hotmail.com,Danielle,Henderson,1970-08-11,7389 Alec Squares Suite 508,Port Jonathan,NM,87320,2006-06-06,None
9,84a51f9a-c07f-4b07-b426-af9f6e6142e4,hannah.smith.66@gmail.com,Hannah,Smith,1966-07-01,230 Donna Street,Lake Adrianstad,MN,56413,2006-06-06,None


## Define fact table schema

In [70]:
# setting table schema for fact table

FACTS_TABLE_METADATA = {
    'tickets': {
        'table_name': 'tickets',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('eticket_num', 'string', mode='REQUIRED'),
            bigquery.SchemaField('confirmation', 'string', mode='REQUIRED'),
            bigquery.SchemaField('ticket_date', 'string', mode='NULLABLE'),
            bigquery.SchemaField('price', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('seat', 'string', mode='NULLABLE'),
            bigquery.SchemaField('status', 'string', mode='NULLABLE'),
            bigquery.SchemaField('origin_iata', 'string', mode='NULLABLE'),
            bigquery.SchemaField('destination_iata', 'string', mode='NULLABLE'),
            bigquery.SchemaField('airline_iata', 'string', mode='NULLABLE'),
            bigquery.SchemaField('passenger_id', 'string', mode='NULLABLE'),
        ],
    },
}

In [71]:
# creating df for ticket fact table

ticket_df = df

ticket_col = ['eticket_num', 'confirmation', 'ticket_date', 'price', 'seat', 'status', 'origin_iata', 'destination_iata', 'airline_iata', 'passenger_email']

# group unique tickets
ticket_df = ticket_df.groupby(ticket_col).all()
ticket_df = ticket_df.reset_index().loc[:, ticket_col]

# join passenger df on email to get passenger id and dropping email column

join_pass = pass_df.reset_index()
join_pass = join_pass[['passenger_email', 'passenger_id']]
join_pass = join_pass.set_index('passenger_email')

join_comp = ticket_df.join(join_pass, on='passenger_email', how='inner')
ticket_df = join_comp.drop(columns='passenger_email')

display(ticket_df)



,eticket_num,confirmation,ticket_date,price,seat,status,origin_iata,destination_iata,airline_iata,passenger_id
0,000-238065-9359,KXDIKE,2022-03-22,752.01,10D,active,JJN,ZCO,NZ,84a51f9a-c07f-4b07-b426-af9f6e6142e4
26,007-431058-7762,MJYSLN,2022-03-24,720.21,29E,active,OAK,SRE,MH,84a51f9a-c07f-4b07-b426-af9f6e6142e4
41,011-433525-9702,5MF358,2022-03-23,988.02,20F,active,ORF,PRG,SJ,84a51f9a-c07f-4b07-b426-af9f6e6142e4
49,013-115939-1473,90IN8Z,2022-03-23,364.47,30B,active,KWL,YNT,QR,84a51f9a-c07f-4b07-b426-af9f6e6142e4
84,025-257697-5811,NT8E2A,2022-03-23,507.59,28F,active,REC,NTE,MW,84a51f9a-c07f-4b07-b426-af9f6e6142e4
...,...,...,...,...,...,...,...,...,...,...
3901,979-413386-4728,FU66JE,2022-03-22,419.36,4E,active,MSY,HEL,CZ,84a51f9a-c07f-4b07-b426-af9f6e6142e4
3920,983-119488-7703,98BV7H,2022-03-22,713.44,28C,active,POS,LGA,NK,84a51f9a-c07f-4b07-b426-af9f6e6142e4
3926,984-245886-5879,KIQDQM,2022-03-24,861.75,30E,active,TYN,CLO,DL,84a51f9a-c07f-4b07-b426-af9f6e6142e4
3934,986-705619-4205,EAH78F,2022-03-22,853.21,18D,active,SJU,SXM,DL,84a51f9a-c07f-4b07-b426-af9f6e6142e4


## Load all data into BQ

In [72]:
# create load function

def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

In [73]:
# loading all tables to bq

# load airlines
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{DIMS_TABLE_METADATA['airlines']['table_name']}"
schema = DIMS_TABLE_METADATA['airlines']['schema']
# load dataframe to bq
load_table(airlines_df, client, table_name, schema)

logger.info(f"loaded airlines")

# load airports
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{DIMS_TABLE_METADATA['airports']['table_name']}"
schema = DIMS_TABLE_METADATA['airports']['schema']
# load dataframe to bq
load_table(airports_df, client, table_name, schema)

logger.info(f"loaded airports")

# load passengers
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{DIMS_TABLE_METADATA['passengers']['table_name']}"
schema = DIMS_TABLE_METADATA['passengers']['schema']
# load dataframe to bq
load_table(pass_df, client, table_name, schema)

logger.info(f"loaded passengers")

table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{FACTS_TABLE_METADATA['tickets']['table_name']}"
schema = FACTS_TABLE_METADATA['tickets']['schema']
# load dataframe to bq
load_table(ticket_df, client, table_name, schema)

logger.info(f"loaded tickets facts")

[INFO ][2023-01-06 14:04:01,191][2964262717:0031] : loading table: 'deb-dev-dw.air_travel.airlines'
[INFO ][2023-01-06 14:04:04,507][2964262717:0036] : loaded 48 rows into deb-dev-dw:air_travel.airlines
[INFO ][2023-01-06 14:04:04,508][1879359360:0009] : loaded airlines
[INFO ][2023-01-06 14:04:04,509][2964262717:0031] : loading table: 'deb-dev-dw.air_travel.airports'
[INFO ][2023-01-06 14:04:08,645][2964262717:0036] : loaded 386 rows into deb-dev-dw:air_travel.airports
[INFO ][2023-01-06 14:04:08,648][1879359360:0017] : loaded airports
[INFO ][2023-01-06 14:04:08,649][2964262717:0031] : loading table: 'deb-dev-dw.air_travel.passengers'
[INFO ][2023-01-06 14:04:12,746][2964262717:0036] : loaded 32 rows into deb-dev-dw:air_travel.passengers
[INFO ][2023-01-06 14:04:12,748][1879359360:0025] : loaded passengers
[INFO ][2023-01-06 14:04:12,750][2964262717:0031] : loading table: 'deb-dev-dw.air_travel.tickets'
[INFO ][2023-01-06 14:04:15,808][2964262717:0036] : loaded 3981 rows into deb-dev